In [ ]:
!git clone https://github.com/oKatanaaa/GCT

In [ ]:
!pip install --upgrade git+https://github.com/oKatanaaa/MakiFlow.git

In [ ]:
%tensorflow_version 1.x

In [ ]:
from GCT.tools import load_cifar10
import matplotlib.pyplot as plt
from makiflow.models.classificator import Classificator, CETrainer
from makiflow.generators.classification import cycle_generator
import tensorflow as tf
from makiflow.layers import *
import numpy as np
%matplotlib inline

In [ ]:
(Xtrain, Ytrain), (Xtest, Ytest), (X_mean, X_std) = load_cifar10()

In [ ]:
BATCH_SIZE = 64

In [ ]:
def block(x, id, in_f, out_f):
    SX = x
    FX = x
    
    FX = BatchNormLayer(D=in_f, name=f'bn_main_{id}_1')(FX)
    FX = ActivationLayer(name=f'act_1_{id}')(FX)
    FX = ConvLayer(kw=3, kh=3, in_f=in_f, out_f=out_f, name=f'conv_main_{id}_1', activation=None)(FX)
    
    FX = BatchNormLayer(D=out_f, name=f'bn_main_{id}_2')(FX)
    FX = ActivationLayer(name=f'act_2_{id}')(FX)
    FX = ConvLayer(kw=3, kh=3, in_f=out_f, out_f=out_f, name=f'conv_main_{id}_2', activation=None)(FX)

    if in_f != out_f:
        SX = BatchNormLayer(D=in_f, name=f'bn_skip_{id}_1')(SX)
        SX = ConvLayer(kw=1, kh=1, in_f=in_f, out_f=out_f, name=f'conv_skip_{id}', activation=None)(SX)
    
    x_sum = SumLayer(name=f'sum_{id}')([FX, SX])
    return x_sum

In [ ]:
inp = InputLayer(input_shape=[BATCH_SIZE, 32, 32, 3], name='input')
x = block(inp, id=1, in_f=3, out_f=32)
x = MaxPoolLayer(name='mp1')(x)
# 16x16
x = block(x, id=2, in_f=32, out_f=64)
x = block(x, id=3, in_f=64, out_f=64)
x = MaxPoolLayer(name='mp2')(x)
# 8x8
x = block(x, id=4, in_f=64, out_f=128)
x = block(x, id=5, in_f=128, out_f=128)
x = MaxPoolLayer(name='mp3')(x)
# 4x4
x = block(x, id=6, in_f=128, out_f=256)
x = block(x, id=7, in_f=256, out_f=256)
x = MaxPoolLayer(name='mp4')(x)
# 2x2
x = FlattenLayer('flat')(x)
# 4*256 = 1024
x = DenseLayer(in_d=1024, out_d=256, name='projection_head')(x)
x = DenseLayer(in_d=256, out_d=10, activation=None, name='classification_head')(x)

In [ ]:
model = Classificator(in_x=inp, out_x=x)
sess = tf.Session()
model.set_session(sess)

In [ ]:
# Setup training

In [ ]:
trainer = CETrainer(model=model, train_inputs=[InputLayer(input_shape=[BATCH_SIZE, 32, 32, 3], name='input')])

In [ ]:
opt = tf.train.AdamOptimizer()

In [ ]:
gen = cycle_generator(Xtrain, Ytrain.reshape(-1), BATCH_SIZE)

In [ ]:
## Train a model with classic architecture for the comparison later

In [ ]:
info1 = trainer.fit_generator(gen, optimizer=opt, epochs=5, iter=100, print_period=10)

In [ ]:
model.evaluate(Xtest, Ytest.reshape(-1))

In [ ]:
from makiflow.core import MakiLayer

In [ ]:
class NewLayer(MakiLayer):    
    def _forward(self, x, computation_mode=MakiLayer.INFERENCE_MODE):
        pass
    
    def _training_forward(self, x):
        pass
    
    @staticmethod
    def build(params: dict):
        pass

    def to_dict(self):
        pass

In [ ]:
def block(x, id, in_f, out_f):
    SX = x
    FX = x
    
    FX = BatchNormLayer(D=in_f, name=f'bn_main_{id}_1')(FX)
    FX = ActivationLayer(name=f'act_1_{id}')(FX)
    FX = ConvLayer(kw=3, kh=3, in_f=in_f, out_f=out_f, name=f'conv_main_{id}_1', activation=None)(FX)
    
    FX = BatchNormLayer(D=out_f, name=f'bn_main_{id}_2')(FX)
    FX = ActivationLayer(name=f'act_2_{id}')(FX)
    FX = ConvLayer(kw=3, kh=3, in_f=out_f, out_f=out_f, name=f'conv_main_{id}_2', activation=None)(FX)

    if in_f != out_f:
        SX = BatchNormLayer(D=in_f, name=f'bn_skip_{id}_1')(SX)
        SX = ConvLayer(kw=1, kh=1, in_f=in_f, out_f=out_f, name=f'conv_skip_{id}', activation=None)(SX)
    
    x_sum = SumLayer(name=f'sum_{id}')([FX, SX])
    return x_sum

In [ ]:
inp = InputLayer(input_shape=[BATCH_SIZE, 32, 32, 3], name='input')
x = block(inp, id=1, in_f=3, out_f=32)
x = MaxPoolLayer(name='mp1')(x)
# 16x16
x = block(x, id=2, in_f=32, out_f=64)
x = block(x, id=3, in_f=64, out_f=64)
x = MaxPoolLayer(name='mp2')(x)
# 8x8
x = block(x, id=4, in_f=64, out_f=128)
x = block(x, id=5, in_f=128, out_f=128)
x = MaxPoolLayer(name='mp3')(x)
# 4x4
x = block(x, id=6, in_f=128, out_f=256)
x = block(x, id=7, in_f=256, out_f=256)
x = MaxPoolLayer(name='mp4')(x)
# 2x2
x = FlattenLayer('flat')(x)
# 4*256 = 1024
x = DenseLayer(in_d=1024, out_d=256, name='projection_head')(x)
x = DenseLayer(in_d=256, out_d=10, activation=None, name='classification_head')(x)

In [ ]:
model2 = Classificator(in_x=inp2, out_x=x2)
model2.set_session(sess)

In [ ]:
opt2 = tf.train.AdamOptimizer()

In [ ]:
trainer2 = CETrainer(model=model2, train_inputs=[InputLayer(input_shape=[BATCH_SIZE, 32, 32, 3], name='input')])

In [ ]:
info1 = trainer2.fit_generator(gen, optimizer=opt2, epochs=5, iter=100, print_period=10)

In [ ]:
plt.plot(info1['test errors'], label='model1 errors')
plt.legend()
plt.plot(info2['test errors'], label='model2 errors')
plt.legend()